**Short introduction to ChIP-seq data analysis**

ChIP-seq (Chromatin immunoprecipitation followed by high-throughput sequencing ) has become the most used technology for the identification of target sites for DNA-binding proteins.

 In this small exercise, we will be working with 2 transcription factors: TATA-binding protein and NF-YA (Nuclear transcription factor Y subunit alpha). We will try to identify binding site in the human genome for these 2 transcription factors using a bioinformatic approach.


A few introductory points:


*   You can run the code in each cell by clicking on the run cell sign, when the code finished running a small green check sign will appear on the left side  
*   You need to run the cells in **sequential order**, please do not run a cell until the one above finished running and do not skip any cells
*   Each cell contains a short description of the code and the output you should get. Please try not to focus on understanding what each command line does, this is not the purpose of the exercise


**Install and import the packages necessary to run this python notebook**
*   Running the cell below will install all the necessary programs that you will  be using, this might take some time, approximately 10 min (Please ignore all the messages you see when installing the various packages)
*   You can use this time to go over the ChiP-seq introduction you were presented in the course

In [ ]:
#Install all the necessary programs and packages 
!sudo apt -qq install bowtie2
!yes | pip install macs3
!sudo apt-get -qq install -y samtools
#!yes | pip install deeptools
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -q
!bash Miniconda3-latest-Linux-x86_64.sh -b -q -p /content/miniconda
!rm /content/Miniconda3-latest-Linux-x86_64.sh
!/content/miniconda/bin/conda config --add channels bioconda
!/content/miniconda/bin/conda config --add channels conda-forge
!/content/miniconda/bin/conda create --name myenv -y -q
!/content/miniconda/bin/conda install -n myenv R -y -q
!/content/miniconda/bin/conda install -n myenv -c bioconda bioconductor-chipseeker bioconductor-txdb.hsapiens.ucsc.hg38.knowngene -q -y

**Download the datasets**

The command lines below will download the raw data you will be working with. The files are stored in a GitHub repository. You will be working with 2 small ChIP-seq experiments. One library for the TATA-binding protein (TBP) and the other for the NF-YA transcription factor. We will be using as reference a small portion of the human genome, only the first 10 Mbp of Chromosome 1.

*   You can check the downloaded files on the left panel, by clicking on "Files" tab and navigating to the data folder.
*   Sequenced reads are saved in .fastq files, while the genome data is stored in a .fasta file.

In [ ]:
!rm -r sample_data/
!rm -f data
!git clone https://github.com/LaviFechete/MBG_Exercises.git data

**Data Analysis**

We will now analyse the data to identify binding sites for our 2 transcription factors.


---


Initial parts of the analysis of sequenced reads include **read alignment, filtering and peak calling**.

> We will use the program called Bowtie2 to align the reads in the 2 .fastq files to our reference sequence (10 Mbp of the human genome). This will help us determine where on the human genome our reads originated from.

> Afterwards, we will keep only the reads that mapped with high quality to the part of the human genome we are interested in. We will use the samtools program for this step.

> The output will be 2 alignment files (.bam). We will use these files in the next steps.




In [ ]:
!mkdir bowtie_index
!bowtie2-build /content/data/NC_000001.11_subset.fasta bowtie_index/Chr1_index

!mkdir bowtie2_results
!bowtie2 -q --local \
-x  bowtie_index/Chr1_index \
-U /content/data/TBP_chr1_subset.fastq \
-S bowtie2_results/TBP_chr1.sam

!bowtie2 -q --local \
-x  bowtie_index/Chr1_index \
-U /content/data/NF-YA_chr1_subset.fastq \
-S bowtie2_results/NF-YA_chr1.sam

!samtools view -q 10 -u bowtie2_results/TBP_chr1.sam | samtools sort -o bowtie2_results/TBP_chr1_sorted.bam
!samtools view -q 10 -u bowtie2_results/NF-YA_chr1.sam | samtools sort -o bowtie2_results/NF-YA_chr1_sorted.bam 
!samtools index bowtie2_results/TBP_chr1_sorted.bam
!samtools index bowtie2_results/NF-YA_chr1_sorted.bam 

**Peak calling**

We can now identify binding sites for the transcription factors, these are also called peaks. These are just regions in the genome where a high number of reads are bound, indicating that the region might be a transcription factor binding site.

> We will use the program macs3 to call peaks for the 2 previously generated alignment files. 


![alt text](https://hbctraining.github.io/Intro-to-ChIPseq/img/plos_chipseq_arrow.png)

In [ ]:
!mkdir macs3_results
!macs3 callpeak -t /content/bowtie2_results/TBP_chr1_sorted.bam -f BAM -g 10000000 --outdir /content/macs3_results/ -m 1 100 -n TBP --verbose 0
!macs3 callpeak -t /content/bowtie2_results/NF-YA_chr1_sorted.bam -f BAM -g 10000000 --outdir /content/macs3_results/ -m 1 100 -n NF-YA --verbose 0

The cell below gives a summary of the number of binding sites identified for each of the 2 transcription factors. 

> Are the number of peaks identified for TBP and NF-YA similar?



In [ ]:
!wc -l /content/macs3_results/*.narrowPeak

**IGV genome browser**

We will now download all the data generated so far and upload it to a genome browser, to visualize the mapped reads and the binding sites for the transcription factors. 

> Run the cell below and pick a place on your computer where to save the .zip file. Find the file on your computer and extract the archive. You should name have a folder named IGV_files, which contains 6 files.

> Open the link: https://igv.org/app/

> Click on the "Tracks" tab and then on "Local file...". Navigate to the place where you saved the "IGV_files" folder on your computer and pick all 6 files in the foler. Click open.


> You can now visualize all the comparatively the ChiPSeq reads mapped to the genome, the gene annotations and the peaks called my macs3. Remember we are only working with the first 10 Mbp of chromosome 1, Zoom in on that region.

There are two tracks, one for NF-YA and one for TBP. Try to identify some of the putative binding sites(regions with a high number of aligned reads) for the transcription factors. 









In [ ]:
#Files for IGV
!mkdir IGV_files
!cp /content/bowtie2_results/NF-YA_chr1_sorted.bam /content/IGV_files/NF-YA_chr1_sorted.bam
!cp /content/bowtie2_results/NF-YA_chr1_sorted.bam.bai /content/IGV_files/NF-YA_chr1_sorted.bam.bai
!cp /content/bowtie2_results/TBP_chr1_sorted.bam /content/IGV_files/TBP_chr1_sorted.bam
!cp /content/bowtie2_results/TBP_chr1_sorted.bam.bai  /content/IGV_files/TBP_chr1_sorted.bam.bai
!cp /content/macs3_results/NF-YA_peaks.narrowPeak /content/IGV_files/NF-YA_peaks.narrowPeak
!cp /content/macs3_results/TBP_peaks.narrowPeak /content/IGV_files/TBP_peaks.narrowPeak

!zip -r IGV_files.zip IGV_files
from google.colab import files
files.download("IGV_files.zip")

We will now do some analyses in order to compare the TBP and NF-YA transcription factors using the

 **ChIPseeker package in R**

Load R 

In [ ]:
%load_ext rpy2.ipython
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)

Import the packages we will be using 

In [ ]:
%%R
.libPaths( c( .libPaths(), "/content/miniconda/envs/myenv/lib/R/library"))
install.packages("graphlayouts")
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

Load the data we previously generated

In [ ]:
%%R
# Load data
TBP <- read.delim("/content/macs3_results/TBP_summits.bed", header=F)
TBP$V1 <- "chr1"
NF_YA <- read.delim("/content/macs3_results/NF-YA_summits.bed", header=F)
NF_YA$V1 <- "chr1"
write.table(TBP, "/content/macs3_results/TBP_renamed.bed", sep="\t", row.names=F, col.names=F, quote=F)
write.table(NF_YA, "/content/macs3_results/NF-YA_renamed.bed", sep="\t", row.names=F, col.names=F, quote=F)
samplefiles <- list("/content/macs3_results/TBP_renamed.bed", "/content/macs3_results/NF-YA_renamed.bed")
names(samplefiles) <- c("TBP", "NF_YA")

In the IGV browser, you visualized the aligned reads and peaks over small regions of the reference sequence. 

Now, we would like to know the peak locations over the whole 10 MBp from the human genome. We can do this by generating a coverage plot. Each vertical line represents a binding site. You can observe that TBP has many more peaks called compared with NF-YA.

In [ ]:
%%R
print(covplot(samplefiles$TBP, weightCol="V5", title = "ChIP Peaks over region - TBP"))
print(covplot(samplefiles$NF_YA, weightCol="V5", title = "ChIP Peaks over region - NF-YA"))

We can next identify the regions to which the transcription factors bind, in terms of genomic features(Promoter, intron, exon etc.). 

In [ ]:
%%R
peakAnnoList <- lapply(samplefiles, annotatePeak, TxDb=txdb, 
                       tssRegion=c(-3000, 3000), verbose=FALSE)
peakAnnoList

As expected a higher proportion of the binding sites is located in the promoter regions. 

We can also create a graphical output with the binding sites annotations. 

In [ ]:
%%R
plotAnnoBar(peakAnnoList)

We've seen that TBP binds to many more sites compared to NF-YA, but let's find out how many binding sites are common for the 2 transcription factors. 

In [ ]:
%%R
genes= lapply(peakAnnoList, function(i) as.data.frame(i)$geneId)
vennplot(genes)